# Introduction

In this post we take a look at the function calling capabilities of the open source model
`NousResearch/Hermes-2-Pro-Mistral-7B`  

(@Hermes-2-Pro-Mistral-7B).


- [Twitter announcement](https://x.com/Teknium1/status/1768023030843015208?s=20)
- [GitHub Repo for Function Calling](https://github.com/NousResearch/Hermes-Function-Calling)
- [Hugging Face Model Card](https://huggingface.co/NousResearch/Hermes-2-Pro-Mistral-7B)


# ENV Setup

Start by creating a virtual environment: 

```
python3 -m venv env
source env/bin/activate
```

Then install:

```
pip install openai
pip install python-dotenv # or define your environment variables differently
pip install langchain # utilities for converting functions to OpenAI tools format.
```

I also have:

- an [OpenAI account](https://platform.openai.com/api-keys) with an API key.
- Hugging Face Account, Access Token, and created [inference endpoint](https://huggingface.co/inference-endpoints/dedicated) with the model `NousResearch/Hermes-2-Pro-Mistral-7B`.

In my `.env` file I have the following:
```
OPENAI_API_KEY=your_key
HUGGING_FACE_ACCESS_TOKEN=your_key
HUGGING_FACE_ENDPOINT_URL=url_for_endpoint
```


In [1]:
# | output: false
import os

from dotenv import load_dotenv

load_dotenv()

HUGGING_FACE_ACCESS_TOKEN = os.environ["HUGGING_FACE_ACCESS_TOKEN"]
HUGGING_FACE_ENDPOINT_URL = os.environ["HUGGING_FACE_ENDPOINT_URL"]

# LLM Inference Class

In a previous [blog post](https://drchrislevy.github.io/posts/llm_inference_class/llm_inference.html) I discussed how we can use the OpenAI python client to run inference with open source models through services that are OpenAI compatible. I'm going to copy part of the code here. 

In [2]:
import ast
import json
import random
from datetime import datetime, timedelta
from typing import Any, Dict, List, Optional, Union

from langchain.tools import tool
from langchain_core.utils.function_calling import convert_to_openai_tool
from openai import OpenAI
from openai._streaming import Stream
from openai.types.chat.chat_completion import ChatCompletion
from openai.types.chat.chat_completion_chunk import ChatCompletionChunk

today = datetime.now().strftime("%A %Y-%m-%d")


class OpenAIChatCompletion:
    clients: Dict = dict()

    @classmethod
    def _load_client(cls, base_url: Optional[str] = None, api_key: Optional[str] = None) -> OpenAI:
        client_key = (base_url, api_key)
        if OpenAIChatCompletion.clients.get(client_key) is None:
            OpenAIChatCompletion.clients[client_key] = OpenAI(base_url=base_url, api_key=api_key)
        return OpenAIChatCompletion.clients[client_key]

    def __call__(
        self,
        model: str,
        messages: list,
        base_url: Optional[str] = None,
        api_key: Optional[str] = None,
        **kwargs: Any,
    ) -> Union[ChatCompletion, Stream[ChatCompletionChunk]]:
        # https://platform.openai.com/docs/api-reference/chat/create
        # https://github.com/openai/openai-python
        client = self._load_client(base_url, api_key)
        return client.chat.completions.create(model=model, messages=messages, **kwargs)

Simply use it like this.

In [3]:
llm = OpenAIChatCompletion()
print(llm(model="gpt-3.5-turbo-0125", messages=[dict(role="user", content="Hello!")]))

ChatCompletion(id='chatcmpl-93FpmWcebHwAX2c8nhtUqjjWiofxt', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello! How can I assist you today?', role='assistant', function_call=None, tool_calls=None))], created=1710562878, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint='fp_4f2ebda25a', usage=CompletionUsage(completion_tokens=9, prompt_tokens=9, total_tokens=18))


We can also use the same code to run inference with `Hermes-2-Pro-Mistral-7B`.
Now, you don't need to use an inference endpoint. You could use the transformers library directly
and run it locally if you want. I'm choosing to run it as a Hugging Face inference endpoint
because it's simple to set up. Remember you have different options for the [prompt format](https://huggingface.co/NousResearch/Hermes-2-Pro-Mistral-7B#prompt-format). I'm using the messages format.

In [4]:
print(
    llm(
        model="tgi",
        api_key=HUGGING_FACE_ACCESS_TOKEN,
        base_url=HUGGING_FACE_ENDPOINT_URL,
        messages=[
            dict(
                role="system",
                content="You are an OpenSource LLM that rivals OpenAI GPT. Your goal in life is to bring open source AI to the masses.",
            ),
            dict(role="user", content="Explain why open source AI is so important."),
        ],
        max_tokens=2000,
        temperature=1,
    )
    .choices[0]
    .message.content
)

Open source AI is crucial for several reasons. 

First, it promotes transparency and collaboration. As the source code is openly available, the AI community can review, critique, and improve the models, leading to better and more reliable results. Collaboration allows diverse perspectives and expertise to contribute to the development of the AI, leading to more innovative solutions.

Second, open source AI encourages accessibility. By being available for free, it democratizes access to advanced AI technologies, enabling researchers, students, and small organizations with limited resources to access and utilize cutting-edge AI models.

Third, it fosters rapid advancement in the AI field. The shared knowledge and resources in open source AI give developers a solid foundation on which to build new models or tailor existing ones to specific needs. This accelerates innovation and allows the field to progress at a faster rate than if proprietary models were dominant.

Finally, open source AI

# Function Calling Capabilities

First we will define some dummy functions/tools which the LLM will have access to.
I use `langchain` here to convert the Python functions to the `tools` format used
by OpenAI. Note that `Hermes-2-Pro-Mistral-7B` can also use this same format!

In [5]:
@tool
def get_weather_forecast(location: str, date: str) -> str:
    """
    Provides a weather forecast for a given location and date.

    Args:
        location (str): The name of the city and state, e.g. 'San Francisco, CA'.
        date (str): The date of the forecast in YYYY-MM-DD format, e.g. '2023-07-01'.

    Returns:
        str: A string containing the weather forecast, e.g. 'Partly cloudy with a high of 72F (22C).'
    """
    pass


@tool
def book_flight(
    departure_city: str,
    arrival_city: str,
    departure_date: str,
    return_date: str,
    num_passengers: int,
    cabin_class: str,
) -> dict:
    """
    Book a round-trip flight for the given parameters.

    Args:
        departure_city (str): The full city name with the departure airport, e.g. "Toronto".
        arrival_city (str): The full city name with the arrival airport, e.g. "Austin".
        departure_date (str): The departure date in YYYY-MM-DD format.
        return_date (str): The return date in YYYY-MM-DD format.
        num_passengers (int): The number of passengers.
        cabin_class (str): The cabin class, e.g. "economy", "business", "first".

    Returns:
        dict: A dict with the booking details including airline, flight numbers, price and booking confirmation code.
    """
    pass


@tool
def book_movie_tickets(movie_name: str, theater_name: str, date: str, time: str, num_tickets: int) -> dict:
    """
    Book movie tickets for the given movie, theater, date, time, and number of tickets.
    Args:
        movie_name (str): The name of the movie.
        theater_name (str): The name of the theater.
        date (str): The date of the movie showing (YYYY-MM-DD).
        time (str): The time of the movie showing (HH:MM).
        num_tickets (int): The number of tickets to book.
    Returns:
        dict: Returns a dictionary with booking details if successful, otherwise returns a dictionary with an error message.
    """
    pass


@tool
def translate_text(text: str, target_language: str) -> str:
    """
    Translate the given text into the specified target language.
    Args:
        text (str): The text to be translated.
        target_language (str): The target language code (e.g., 'es' for Spanish, 'fr' for French).
    Returns:
        str: The translated text in the target language.
    """
    pass


@tool
def get_recipe(dish_name: str) -> str:
    """
    Returns a recipe for the given dish name, optionally filtered by cuisine type and dietary restrictions.
    Args:
        dish_name (str): The name of the dish to get the recipe for.
    Returns:
        str: A string containing the recipe instructions.
    """
    pass


@tool
def solve_math_problem(problem: str) -> str:
    """
    Solves a given math problem using a symbolic math library or calculator API.
    Args:
        problem (str): The math problem to solve, expressed as a string.
                       diff(f(x),x) # derivative
                       integrate(f(x), x) # integrate
                       solve(3x=2,x) # solve for x
    Returns:
        str: The solution to the math problem.
    """
    pass


@tool
def send_slack_message(channel_name: str, message: str) -> bool:
    """
    Send a message to a Slack channel.
    Args:
        channel_name (str): The name of the channel.
        message (str): The message to be sent.
    Returns:
        bool: True if the message was sent successfully, False otherwise.
    """
    pass


functions = [
    get_weather_forecast,
    book_flight,
    book_movie_tickets,
    translate_text,
    get_recipe,
    solve_math_problem,
    send_slack_message,
]
tools = [convert_to_openai_tool(f) for f in functions]

In [6]:
tools[0]

{'type': 'function',
 'function': {'name': 'get_weather_forecast',
  'description': "get_weather_forecast(location: str, date: str) -> str - Provides a weather forecast for a given location and date.\n\n    Args:\n        location (str): The name of the city and state, e.g. 'San Francisco, CA'.\n        date (str): The date of the forecast in YYYY-MM-DD format, e.g. '2023-07-01'.\n\n    Returns:\n        str: A string containing the weather forecast, e.g. 'Partly cloudy with a high of 72F (22C).'",
  'parameters': {'type': 'object',
   'properties': {'location': {'type': 'string'}, 'date': {'type': 'string'}},
   'required': ['location', 'date']}}}

In [7]:
tools[-1]

{'type': 'function',
 'function': {'name': 'send_slack_message',
  'description': 'send_slack_message(channel_name: str, message: str) -> bool - Send a message to a Slack channel.\n    Args:\n        channel_name (str): The name of the channel.\n        message (str): The message to be sent.\n    Returns:\n        bool: True if the message was sent successfully, False otherwise.',
  'parameters': {'type': 'object',
   'properties': {'channel_name': {'type': 'string'},
    'message': {'type': 'string'}},
   'required': ['channel_name', 'message']}}}

Here is a list of questions to test out the function calling. For each question
we have the text and the ground truth i.e. expected function/tool name and arguments.
This way we can have a mini evaluation for how well the function calling works.

In [8]:
questions = [
    {
        "question": "What will the weather be like in Seattle, WA tomorrow?",
        "tool_calls": [
            {
                "name": "get_weather_forecast",
                "arguments": {
                    "location": "Seattle, WA",
                    "date": (datetime.now() + timedelta(days=1)).strftime("%Y-%m-%d"),
                },
            }
        ],
    },
    {
        "question": "What's the forecast for Miami for today?",
        "tool_calls": [
            {
                "name": "get_weather_forecast",
                "arguments": {"location": "Miami, FL", "date": datetime.now().strftime("%Y-%m-%d")},
            }
        ],
    },
    {
        "question": "Will I need an umbrella in New York City two days from now?",
        "tool_calls": [
            {
                "name": "get_weather_forecast",
                "arguments": {
                    "location": "New York City, NY",
                    "date": (datetime.now() + timedelta(days=2)).strftime("%Y-%m-%d"),
                },
            }
        ],
    },
    {
        "question": "Book me a round-trip flight from New York City to Los Angeles departing on June 15th and returning June 22nd for 2 passengers in economy class.",
        "tool_calls": [
            {
                "name": "book_flight",
                "arguments": {
                    "departure_city": "NYC",
                    "arrival_city": "LAX",
                    "departure_date": datetime(datetime.now().year, 6, 15).strftime("%Y-%m-%d"),
                    "return_date": datetime(datetime.now().year, 6, 22).strftime("%Y-%m-%d"),
                    "num_passengers": 2,
                    "cabin_class": "economy",
                },
            }
        ],
    },
    {
        "question": "I need to book a first class round-trip flight for 4 people from Chicago to Miami. We want to leave on December 1 and return on December 12.",
        "tool_calls": [
            {
                "name": "book_flight",
                "arguments": {
                    "departure_city": "Chicago",
                    "arrival_city": "Miami",
                    "departure_date": datetime(datetime.now().year, 12, 1).strftime("%Y-%m-%d"),
                    "return_date": datetime(datetime.now().year, 12, 12).strftime("%Y-%m-%d"),
                    "num_passengers": 4,
                    "cabin_class": "first",
                },
            }
        ],
    },
    {
        "question": "I want to book 3 tickets for The Super Mario Bros. Movie at AMC Empire 25 on April 7th at 7:30 PM.",
        "tool_calls": [
            {
                "name": "book_movie_tickets",
                "arguments": {
                    "movie_name": "The Super Mario Bros. Movie",
                    "theater_name": "AMC Empire 25",
                    "date": datetime(datetime.now().year, 4, 7).strftime("%Y-%m-%d"),
                    "time": "19:30",
                    "num_tickets": 3,
                },
            }
        ],
    },
    {
        "question": "Book 2 tickets for Guardians of the Galaxy Vol. 3 at Regal Union Square on May 5th for the 9:45 PM show.",
        "tool_calls": [
            {
                "name": "book_movie_tickets",
                "arguments": {
                    "movie_name": "Guardians of the Galaxy Vol. 3",
                    "theater_name": "Regal Union Square",
                    "date": datetime(datetime.now().year, 5, 5).strftime("%Y-%m-%d"),
                    "time": "21:45",
                    "num_tickets": 2,
                },
            }
        ],
    },
    {
        "question": "How do you say 'Hello, how are you?' in Spanish?",
        "tool_calls": [
            {
                "name": "translate_text",
                "arguments": {"text": "Hello, how are you?", "target_language": "es"},
            }
        ],
    },
    {
        "question": "Translate 'I love programming' to French.",
        "tool_calls": [
            {
                "name": "translate_text",
                "arguments": {"text": "I love programming", "target_language": "fr"},
            }
        ],
    },
    {
        "question": "How do I make pesto?",
        "tool_calls": [{"name": "get_recipe", "arguments": {"dish_name": "pesto"}}],
    },
    {
        "question": "What's a good vegan chili recipe?",
        "tool_calls": [{"name": "get_recipe", "arguments": {"dish_name": "vegan chili"}}],
    },
    {
        "question": "Can you give me a recipe for chocolate chip cookies?",
        "tool_calls": [{"name": "get_recipe", "arguments": {"dish_name": "chocolate chip cookies"}}],
    },
    {
        "question": "What is the result of integrating x^2 + 2x + 1 with respect to x?",
        "tool_calls": [{"name": "solve_math_problem", "arguments": {"problem": "integrate(x^2 + 2x + 1, x)"}}],
    },
    {
        "question": "Solve the equation: 3x - 7 = 5x + 9",
        "tool_calls": [{"name": "solve_math_problem", "arguments": {"problem": "solve(3x - 7 = 5x + 9, x)"}}],
    },
    {
        "question": "Calculate the derivative of sin(x^3) with respect to x.",
        "tool_calls": [{"name": "solve_math_problem", "arguments": {"problem": "diff(sin(x^3), x)"}}],
    },
    {
        "question": "Send a message to the general channel on Slack saying 'Hello, world!'",
        "tool_calls": [
            {
                "name": "send_slack_message",
                "arguments": {"channel_name": "general", "message": "Hello, world!"},
            }
        ],
    },
    {
        "question": "Send a message to the sales-team channel on Slack telling them to check their email",
        "tool_calls": [
            {
                "name": "send_slack_message",
                "arguments": {
                    "channel_name": "sales-team",
                    "message": "Please check your email.",
                },
            }
        ],
    },
    {
        "question": "Send a message to the office-updates channel on Slack telling them to the food is here. ",
        "tool_calls": [
            {
                "name": "send_slack_message",
                "arguments": {"channel_name": "office-updates", "message": "The food is here!"},
            }
        ],
    },
]

In [9]:
random.shuffle(tools)
random.shuffle(questions)

## gpt-3.5-turbo-0125 Function Calling

First we will use `gpt-3.5-turbo-0125` to extract the function name and arguments for each question.

In [10]:
def extract_tool_calls(resp):
    resp = resp.choices[0].message
    if resp.tool_calls:
        final_tools = []
        for tool_call in resp.tool_calls:
            final_tools.append(
                {
                    "name": tool_call.function.name,
                    "arguments": json.loads(tool_call.function.arguments),
                }
            )
        return final_tools
    else:
        return None

I'm also going to use GPT4 to check the "correctness" of the expected
function arguments and the predicted/generated function arguments. 
It's not perfect but good enough.

In [11]:
def check_tool_call_arguments(expected, predicted):
    # Ask GPT4 if the expected arguments and predicted arguments are "the same".
    if expected["name"] != predicted["name"]:
        return False
    prompt = f"""
Are the following queries approx equal. Use fuzzy matching for strings. Just looking for semantic similar. 
If so then return TRUE and only TRUE with no other explanation. 
Otherwise FALSE and explain.

Expected Arguments: {expected['arguments']}
Predicted Arguments: {predicted['arguments']}
    """
    resp = llm(model="gpt-4-0125-preview", messages=[dict(role="user", content=prompt)])
    if resp.choices[0].message.content.lower().strip() == "true":
        return True, None
    explanation = resp.choices[0].message.content.lower().strip()
    return False, explanation

Okay, let's loop over the questions and use `gpt-3.5-turbo-0125` to extract the function name and arguments,
and keep track of the results.

In [ ]:
total = 0
total_correct = 0
for question in questions:
    resp = llm(
        model="gpt-3.5-turbo-0125",
        tools=tools,
        messages=[
            dict(role="system", content=f"The date today is {today}"),
            dict(role="user", content=question["question"]),
        ],
    )
    tool_calls = extract_tool_calls(resp)
    assert len(tool_calls) == len(question["tool_calls"])
    for tool_call, expected_call in zip(tool_calls, question["tool_calls"]):
        correct_call, explanation = check_tool_call_arguments(expected_call, tool_call)
        if not correct_call:
            print(f'QUESTION: {question["question"]}')
            print(f'EXPECTED Tool Call: {question["tool_calls"][0]}')
            print(f"GENERATED Tool Call: {tool_call}")
            print(f"EXPLANATION: {explanation}\n\n")
        else:
            total_correct += 1
        total += 1

QUESTION: Send a message to the office-updates channel on Slack telling them to the food is here. 
EXPECTED Tool Call: {'name': 'send_slack_message', 'arguments': {'channel_name': 'office-updates', 'message': 'The food is here!'}}
GENERATED Tool Call: {'name': 'send_slack_message', 'arguments': {'channel_name': 'office-updates', 'message': 'The food is here! Enjoy your meal 🍕🥗'}}
EXPLANATION: false 

the key 'channel_name' matches exactly in both arguments, which is semantically similar. however, the 'message' key, while quite similar, includes the additional phrase "enjoy your meal 🍕🥗" in the predicted arguments. this extension changes the semantic content by not only announcing the arrival of food but also extending a wish/enjoyment aspect with emojis suggesting the type of food. therefore, semantically, they are not equivalent due to the added context in the predicted argument.



In [ ]:
print(
    f'Correctly called the proper functions {total_correct} times out of {total}. But check the "failure" cases above since they may be correct anyway.'
)

## NousResearch/Hermes-2-Pro-Mistral-7B Function Calling

Now we are going to do the same thing with `NousResearch/Hermes-2-Pro-Mistral-7B`.
The format for the function calling is documented 
on the [model card](https://huggingface.co/NousResearch/Hermes-2-Pro-Mistral-7B#prompt-format-for-function-calling) as well as this [repo](https://github.com/NousResearch/Hermes-Function-Calling). 

In [ ]:
def extract_tool_calls(tool_calls_str):
    # Split the string by "</tool_call>\n" to separate each tool call
    tool_calls = tool_calls_str.split("</tool_call>\n")
    parsed_results = []
    for tool_call in tool_calls:
        if tool_call:
            dict_str = tool_call.split("\n")[1]  # Get the line with the dictionary
            tool_call_dict = ast.literal_eval(dict_str)
            # Extracting the arguments and function name and add to the results list
            parsed_results.append({"arguments": tool_call_dict["arguments"], "name": tool_call_dict["name"]})
    return parsed_results


system_prompt = (
    f"The date today is {today}\n"
    + """
You are a function calling AI model. You are provided with function signatures within <tools></tools> XML tags. You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug into functions. Here are the available tools:
<tools> 
"""
    + str(tools)
    + """
    
</tools> Use the following pydantic model json schema for each tool call you will make: {'title': 'FunctionCall', 'type': 'object', 'properties': {'arguments': {'title': 'Arguments', 'type': 'object'}, 'name': {'title': 'Name', 'type': 'string'}}, 'required': ['arguments', 'name']} For each function call return a json object with function name and arguments within <tool_call></tool_call> XML tags as follows:
<tool_call>
{'arguments': <args-dict>, 'name': <function-name>}
</tool_call>
"""
)

total = 0
total_correct = 0
for question in questions:
    resp = llm(
        model="tgi",
        base_url=HUGGING_FACE_ENDPOINT_URL,
        api_key=HUGGING_FACE_ACCESS_TOKEN,
        messages=[
            dict(role="system", content=system_prompt),
            dict(role="user", content=question["question"]),
        ],
    )
    tool_calls = extract_tool_calls(resp.choices[0].message.content)
    assert len(tool_calls) == len(question["tool_calls"])
    for tool_call, expected_call in zip(tool_calls, question["tool_calls"]):
        correct_call, explanation = check_tool_call_arguments(expected_call, tool_call)
        if not correct_call:
            print(f'QUESTION: {question["question"]}')
            print(f'EXPECTED Tool Call: {question["tool_calls"][0]}')
            print(f"GENERATED Tool Call: {tool_call}")
            print(f"EXPLANATION: {explanation}\n\n")
        else:
            total_correct += 1
        total += 1

In [ ]:
print(
    f'Correctly called the proper functions {total_correct} times out of {total}. But check the "failure" cases above since they may be correct anyway.'
)